# Things to do 
# 1. Choose a good cost function and activation function
# 2. Get more data
# 3. The intialization is very important! Some times it will stuck in a local minima, and get a bad training result
# 4. Tuning is very important! Do not use a too complicated CNN for this 1 output model.


In [3]:
import tensorflow as tf
import numpy as np

%matplotlib inline
import pylab

import matplotlib.pyplot as plt
import scipy
import scipy.io as sio
from sklearn.model_selection import train_test_split

image_by_pixel = sio.loadmat('combined_img.mat')
label_by_pixel = sio.loadmat('combined_lab.mat')

test_image_by_pixel = sio.loadmat('s3d2_jpgimage.mat')
test_label_by_pixel = sio.loadmat('s3d2_jpglabel.mat')

In [4]:
def image_process(image, label, down_sample_rate=4):
    images = image['allimages'][::down_sample_rate ,::down_sample_rate , :]  # downsample the image
    labels = label['disc'][::down_sample_rate ,::down_sample_rate , :]       # downsample the pixel
    image_length1 = images.shape[0]
    image_length2 = images.shape[1]
    num_image = images.shape[2]                # number of images
    image_size = image_length1*image_length2   # image size 
    label_whole = np.zeros((num_image, output_size))#the label of whether the image is all trash or not
    images_processed = np.zeros((num_image, image_size))    
    for _ in range(num_image):
        label_whole[_] = 1*(np.linalg.norm(labels[:,:,_]) != 0) 
        images_processed[_] = images[:,:,_].reshape(image_size)
    return images_processed, label_whole, image_size, image_length1


output_size = 1                            # output portal size 1: 0 - garbge 1 - useful

train_image, train_label, image_size, image_length = image_process(image_by_pixel, label_by_pixel, 4)
test_image, test_label, _, _ =  image_process(test_image_by_pixel, test_label_by_pixel, 4)



In [5]:
# define variable placeholder for tf
x = tf.placeholder('float',[None, image_size])
y = tf.placeholder('float', [None, output_size])

In [6]:
############### TUNING PARAMETER ###################
conv1_patch_len = 5
conv1_num_filters = 32

conv2_patch_len = 5
conv2_num_filters = 16

full_con_len = 8
##################################################

def conv2d(x,W):             # strides = 1
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding = 'SAME')

def maxpool2d(x):            #size of window    # movement of window
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
    #return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1])

def conv_neural_net(x):
    weights = {'W_conv1': tf.Variable(tf.random_normal([conv1_patch_len, conv1_patch_len, 1 ,conv1_num_filters], mean=0.0,stddev=0.02)),
               'W_conv2': tf.Variable(tf.random_normal([conv2_patch_len, conv2_patch_len, conv1_num_filters,conv2_num_filters], mean=0.0,stddev=0.02)),
               'W_fc' : tf.Variable(tf.random_normal([image_size//16*conv2_num_filters, full_con_len], mean=0.0,stddev=0.02)),
               'out' : tf.Variable(tf.random_normal([full_con_len ,output_size], mean=0.0,stddev=0.02))}
    
    biases = {'B_conv1': tf.Variable(tf.random_normal([conv1_num_filters], mean=0.0,stddev=0.02)),
               'B_conv2': tf.Variable(tf.random_normal([conv2_num_filters ], mean=0.0,stddev=0.02)),
               'B_fc' : tf.Variable(tf.random_normal([full_con_len], mean=0.0,stddev=0.02)),
               'out' : tf.Variable(tf.random_normal([output_size], mean=0.0,stddev=0.02))}
    
    x = tf.reshape(x, shape = [-1, image_length,image_length, 1])
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1'])+ biases['B_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2'])+ biases['B_conv2'])
    conv2 = maxpool2d(conv2)
    
    fc = tf.reshape(conv2, [-1, image_size//16*conv2_num_filters ])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['B_fc'])
    
    output = tf.matmul(fc, weights['out']+biases['out'])

    return output

In [7]:
def train_neural_network(x):
    prediction = conv_neural_net(x)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=prediction, labels=y) )
    # double check what cost function to use
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    ############### TUNING PARAMETER ###################
    hm_epochs = 50 # number of epochs
    ####################################################
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            _, epoch_loss = sess.run([optimizer, cost], feed_dict={x: train_image, y: train_label})
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
            

        pred =tf.to_int32(tf.greater(prediction, tf.ones([tf.size(y) ,1])), name='ToInt32')
        correct = tf.equal(pred , tf.to_int32(y, name='ToInt32'))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float')) 
        print(prediction.eval({x:test_image}), test_label)
        print('Training Accuracy:',accuracy.eval({x:train_image, y:train_label}))
        print('Test Accuracy:',accuracy.eval({x:test_image, y:test_label}))
        print('y size:', tf.size(y).eval({ y:test_label}))

In [8]:
train_neural_network(x)

Epoch 1 completed out of 50 loss: 0.648726
Epoch 2 completed out of 50 loss: 0.426387
Epoch 3 completed out of 50 loss: 0.447923
Epoch 4 completed out of 50 loss: 0.39197
Epoch 5 completed out of 50 loss: 0.396629
Epoch 6 completed out of 50 loss: 0.366918
Epoch 7 completed out of 50 loss: 0.334578
Epoch 8 completed out of 50 loss: 0.324974
Epoch 9 completed out of 50 loss: 0.297987
Epoch 10 completed out of 50 loss: 0.269995
Epoch 11 completed out of 50 loss: 0.258011
Epoch 12 completed out of 50 loss: 0.241705
Epoch 13 completed out of 50 loss: 0.223438
Epoch 14 completed out of 50 loss: 0.209654
Epoch 15 completed out of 50 loss: 0.199934
Epoch 16 completed out of 50 loss: 0.185919
Epoch 17 completed out of 50 loss: 0.174909
Epoch 18 completed out of 50 loss: 0.166924
Epoch 19 completed out of 50 loss: 0.157736
Epoch 20 completed out of 50 loss: 0.157867
Epoch 21 completed out of 50 loss: 0.150513
Epoch 22 completed out of 50 loss: 0.149996
Epoch 23 completed out of 50 loss: 0.1483
